In [1]:
using Pkg

Pkg.activate(mktempdir())
Pkg.update()

Pkg.add([
    "Yao",
    "YaoPlots",
    "CairoMakie",
    "LinearAlgebra",
    "Statistics",
    "Random",
    "SparseArrays"
])

  Activating new project at `C:\Users\79021\AppData\Local\Temp\jl_srtyR4`
    Updating registry at `C:\Users\79021\.julia\registries\General.toml`
     Project No packages added to or removed from `C:\Users\79021\AppData\Local\Temp\jl_srtyR4\Project.toml`
    Manifest No packages added to or removed from `C:\Users\79021\AppData\Local\Temp\jl_srtyR4\Manifest.toml`
   Resolving package versions...
   Installed YaoToEinsum ─────── v0.2.9
   Installed PolygonAlgorithms ─ v0.3.5
   Installed WoodburyMatrices ── v1.1.0
   Installed Combinatorics ───── v1.1.0
   Installed YaoSym ──────────── v0.6.12
   Installed Graphs ──────────── v1.13.3
    Updating `C:\Users\79021\AppData\Local\Temp\jl_srtyR4\Project.toml`
  [13f3f980] + CairoMakie v0.15.8
  [10745b16] + Statistics v1.11.1
  [5872b779] + Yao v0.9.2
  [32cfe2d9] + YaoPlots v0.9.6
  [37e2e46d] + LinearAlgebra v1.12.0
  [9a3f8284] + Random v1.11.0
  [2f01184e] + SparseArrays v1.12.0
    Updating `C:\Users\79021\AppData\Local\Temp\jl_srtyR4\M

In [2]:
using Yao
using YaoPlots
using CairoMakie
using LinearAlgebra
using Statistics
using Random
using SparseArrays

Random.seed!(42)

TaskLocalRNG()

In [4]:
# ============================================================================
# SECTION 1: CLASSICAL RECAP — The Bottleneck
# ============================================================================
# Classical linear regression solves:
#   w = (XᵀX)⁻¹ Xᵀy
#
# COMPUTATIONAL COST:
# - Matrix multiplication XᵀX: O(mn²) where m = samples, n = features
# - Matrix inversion (XᵀX)⁻¹: O(n³)
# - For large n, this becomes prohibitive
#
# QUANTUM PROMISE:
# HHL algorithm can solve Ax = b in O(log(N) · κ² · 1/ε)
# where N = matrix dimension, κ = condition number, ε = error
# This is exponentially faster in N for sparse/structured matrices!
# ============================================================================

m_classical = 100
n_classical = 4
X_classical = randn(m_classical, n_classical)
X_aug = hcat(X_classical, ones(m_classical))
w_true = [1.5, -2.0, 0.8, 0.5, 1.0]
y_classical = X_aug * w_true .+ 0.1 .* randn(m_classical)

gram_matrix = X_aug' * X_aug
w_classical = gram_matrix \ (X_aug' * y_classical)

println("Section 1: Classical recap")
println("Gram matrix (XᵀX) size: $(size(gram_matrix))")
println("Condition number κ(XᵀX): $(cond(gram_matrix))")
println("Classical weights: $w_classical")

Section 1: Classical recap
Gram matrix (XᵀX) size: (5, 5)
Condition number κ(XᵀX): 2.2967856564666635
Classical weights: [1.482632680390675, -1.9905882660235659, 0.7927736063007289, 0.48696053090236424, 1.00483891884268]


In [5]:
# ============================================================================
# SECTION 2: AMPLITUDE ENCODING — Data as Quantum States
# ============================================================================
# CORE IDEA:
# Classical vector b = [b₁, b₂, ..., bₙ] with ‖b‖ = 1
# Quantum state |b⟩ = Σᵢ bᵢ|i⟩
#
# For N = 2ⁿ dimensional vector, we need only n qubits!
# Example: 1 million dimensions → ~20 qubits
#
# DERIVATION (Schuld 7.1.1):
# Training set of M vectors of dimension N can be represented
# in log₂(MN) qubits via amplitude encoding
# ============================================================================

function amplitude_encode(vec::Vector{<:Real})
    normalised = vec / norm(vec)
    n_qubits = Int(ceil(log2(length(normalised))))
    N = 2^n_qubits

    padded = zeros(N)
    padded[1:length(normalised)] = normalised

    reg = ArrayReg(ComplexF64.(padded))
    return reg, n_qubits
end

test_vec = [0.5, 0.5, 0.5, 0.5]
reg_encoded, n_qubits = amplitude_encode(test_vec)

println("\nSection 2: Amplitude encoding")
println("Classical vector: $test_vec")
println("Qubits needed: $n_qubits")
println("Quantum state amplitudes: $(state(reg_encoded))")

classical_storage = sizeof(Float64) * length(test_vec)
quantum_qubits = n_qubits
println("Classical storage: $classical_storage bytes")
println("Quantum qubits: $quantum_qubits")


Section 2: Amplitude encoding
Classical vector: [0.5, 0.5, 0.5, 0.5]
Qubits needed: 2
Quantum state amplitudes: ComplexF64[0.5 + 0.0im; 0.5 + 0.0im; 0.5 + 0.0im; 0.5 + 0.0im;;]
Classical storage: 32 bytes
Quantum qubits: 2


In [6]:
# ============================================================================
# SECTION 3: QUANTUM FOURIER TRANSFORM (QFT)
# ============================================================================
# QFT transforms computational basis to frequency basis
# |j⟩ → (1/√N) Σₖ exp(2πijk/N)|k⟩
#
# CIRCUIT STRUCTURE:
# For n qubits, apply to each qubit i:
#   1. Hadamard gate H
#   2. Controlled phase rotations R_k from qubits i+1 to n
#   where R_k = diag(1, exp(2πi/2^k))
#
# COMPLEXITY: O(n²) gates for n qubits
# Classical FFT: O(N log N) = O(n · 2ⁿ) operations
# ============================================================================

function build_qft(n::Int)
    circuit = chain(n)
    for i in 1:n
        push!(circuit, put(n, i => H))
        for j in (i+1):n
            k = j - i + 1
            push!(circuit, control(n, j, i => shift(2π / (1 << k))))
        end
    end
    return circuit
end

n_qft = 3
qft_circuit = build_qft(n_qft)

println("\nSection 3: Quantum fourier transform")
println("QFT circuit for $n_qft qubits:")

input_state = zero_state(n_qft)
output_state = copy(input_state) |> qft_circuit

println("Input |000⟩ amplitudes: $(real.(state(input_state)))")
println("Output QFT|000⟩ amplitudes: $(real.(state(output_state)))")


Section 3: Quantum fourier transform
QFT circuit for 3 qubits:
Input |000⟩ amplitudes: [1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
Output QFT|000⟩ amplitudes: [0.3535533905932737; 0.3535533905932737; 0.3535533905932737; 0.3535533905932737; 0.3535533905932737; 0.3535533905932737; 0.3535533905932737; 0.3535533905932737;;]


In [8]:
# ============================================================================
# SECTION 4: QUANTUM PHASE ESTIMATION (QPE)
# ============================================================================
# PROBLEM: Given unitary U and eigenstate |u⟩ with U|u⟩ = e^(2πiφ)|u⟩
#          Estimate the phase φ
#
# ALGORITHM:
# 1. Prepare |0⟩^⊗n |u⟩ (n ancilla qubits + eigenstate)
# 2. Apply Hadamards to ancillas: (1/√2ⁿ) Σⱼ|j⟩|u⟩
# 3. Apply controlled-U^(2^k) operations:
#    (1/√2ⁿ) Σⱼ e^(2πiφj)|j⟩|u⟩
# 4. Apply inverse QFT to ancillas
# 5. Measure ancillas to get φ in binary
#
# RESULT: |φ̃⟩ where φ̃ is n-bit approximation of φ
# ============================================================================

function simulate_qpe(eigenvalue_phase, n_ancilla)
    N = 2^n_ancilla
    
    phases = [k * eigenvalue_phase for k in 0:(N-1)]

    state_after_ctrl_U = exp.(2π * im .* phases) / sqrt(N)

    F = [exp(-2π * im * j * k / N) / sqrt(N) for j in 0:(N-1), k in 0:(N-1)]
    final_state = F * state_after_ctrl_U

    probabilities = abs2.(final_state)
    measured_idx = argmax(probabilities) - 1
    estimated_phase = measured_idx / N
    
    return estimated_phase, probabilities
end

true_phase = 0.25
n_ancilla = 4
est_phase, probs = simulate_qpe(true_phase, n_ancilla)

println("\nSection 4: Quantum phase estimation")
println("True phase: $true_phase")
println("Estimated phase: $est_phase")
println("Estimated error: $(abs(true_phase - est_phase))")


Section 4: Quantum phase estimation
True phase: 0.25
Estimated phase: 0.25
Estimated error: 0.0
